In [ ]:
import pandas as pd
import numpy as np
import param
import panel
import plotly.express as px
panel.extension('plotly')

In [ ]:
class GenericModel:
    def predict(self, value):
        return np.cos(np.array(range(150))/value)


generic_model = GenericModel()
prediction = generic_model.predict(5)

In [ ]:
class InteractiveDashboard(param.Parameterized):
    denominator_param = param.Number(default=10.0, bounds=(1.0, 50.0), step=1.0, label='test_label')
    useless_parameter = param.String(default='u dont see me', precedence=-1)
    indirect_param = param.Integer(default=150)

    def plot_data(self):
        if self.denominator_param > 40:
            raise ValueError('lol')
        y_values = generic_model.predict(self.denominator_param)
        x_values = np.array(range(0, 150))
        data_df = pd.DataFrame({
            'x_values': x_values,
            'y_values': y_values,
        })
        fig = px.scatter(data_df, x='x_values', y='y_values')
        return fig

interactive_dashboard = InteractiveDashboard()
dashboard = panel.Column(
    'test_title',
    interactive_dashboard.param,
    panel.Param(interactive_dashboard.param, name='test_rename_param'),
    interactive_dashboard.plot_data,
)
panel.Param.display_threshold = 0  # If precedence is lower than this, will not show
dashboard.embed(max_opts=50)

In [ ]:
class GoogleMapViewer(param.Parameterized):

    continent = param.ObjectSelector(default='Asia', objects=['Africa', 'Asia', 'Europe'])
    country = param.ObjectSelector(default='China', objects=['China', 'Thailand', 'Japan'])

    _countries = {'Africa': ['Ghana', 'Togo', 'South Africa', 'Tanzania'],
                  'Asia'  : ['China', 'Thailand', 'Japan'],
                  'Europe': ['Austria', 'Bulgaria', 'Greece', 'Portugal', 'Switzerland']}

    @param.depends('continent', watch=True)
    def _update_countries(self):
        countries = self._countries[self.continent]
        self.param['country'].objects = countries
        # self.country.objects = countries
        self.country = countries[0]

    # @param.depends('country')
    def view(self):
        iframe = """
        <iframe width="800" height="400" src="https://maps.google.com/maps?q={country}&z=6&output=embed"
        frameborder="0" scrolling="no" marginheight="0" marginwidth="0"></iframe>
        """.format(country=self.country)
        # return panel.pane.HTML(iframe, height=400)
        return panel.pane.Plotly('test')

viewer = GoogleMapViewer(name='Google Map Viewer')
map_viewer = panel.Row(
    viewer.param,
    viewer.view,
)
map_viewer.embed(max_opts=20)